# Make OlthoPhoto and PhaseAngleImage

In [16]:
import csv
import cv2
import math
import tifffile
import pandas as pd
import numpy as np
from numpy import loadtxt
import xml.etree.ElementTree as ET
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
import colorsys
import time
from tqdm import tqdm

# sun
import ephem   
import pyproj
from pyproj import Transformer

# exif
import piexif
import re

# Parallel processing
import os
import concurrent.futures as confu

# Initial setting

In [17]:
# Observation scene
SceneID    = "Kotogahama-20231228-1153-P1"
CameraName = "P1"

# Search area (l) in the original image / PixelArea = l 
a              = 7
PixelArea      = a*2+1
ImageExtension = "tif" # original image extensions

# I/O Path
Dir1   = "I_InputFiles/"
Dir2   = "I_Raw/"
Dir3   = "O_Tiff/"
Dir4   = "O_LuminunceImage/"
Dir5   = "O_AngleImage/"

# I/O files
IFile1 = Dir1 +SceneID+ "_CameraOmegaPhiKappa.txt"  # Camera position and orientation information
IFile2 = Dir1 +SceneID+ "_CameraXML.xml"            # Camera sensor information
IFile3 = Dir1 +SceneID+ "_DSMtliming.csv"           # DSM
IFile4 = Dir1 +SceneID+ "_ListImages.csv"           # List of images
OFile1 = Dir1 +SceneID+ "_ListImages-INSIDE.csv"    # List of images within ROI range

# Original Images list
df = pd.read_csv(IFile4, header=None, encoding='utf8')
df.to_csv(OFile1, index=False, header=False)

# Setting the ROI (unit:m)　#############################################
xmin,xmax,ymin,ymax,zmin,zmax = 26251,26320 , 57320,57379 , 30,40
Interval                      =  0.05  # Ground sample distance (unit:m)
IntervalID                    = "005-0"+ str(PixelArea)
xsize                         = int(round((xmax-xmin)/Interval))
ysize                         = int(round((ymax-ymin)/Interval))

#　Camera parameters　　　　　#############################################
camera_data_pd = pd.read_csv(IFile1, skiprows=2, sep='\t', names=("#PhotoID","X","Y","Z","Omega","Phi","Kappa","r11","r12","r13","r21","r22","r23","r31","r32","r33"))

# DSM as the projection surface　########################################
P_pd1 = pd.read_csv(IFile3, skiprows=1, names=("x","y","z","r","g","b","n","i","j","k"))
P_pd2 = P_pd1.drop(["r","g","b","n"],axis=1)
P_np  = P_pd2.values

# All original image processing
PhotoIDs = []
with open(IFile4, "r", encoding="utf-8-sig") as f:
    Photolist = f.readlines()
for i in Photolist:
    PhotoIDs.append(i.rstrip())


print("Number of images:",len(PhotoIDs),",  ","Size of DSM side:",xsize,",  ","Size of DSM Vertical",ysize,)

Number of images: 1 ,   Size of DSM side: 1380 ,   Size of DSM Vertical 1180


## Class & Function

## Sensors Class

In [18]:
class SensorParameter:
    def __init__(self,Sensor):

        self.SensorID = Sensor
        XML = ET.parse(IFile2)
        xml = XML.getroot()
        
        pixel_size        = [str(part.get('value')) for part in xml.iter('property')]
        self.pixel_size   = float(pixel_size[0]) / 1000                  # The physical size of one pixel(unit:m)
        
        cx                = [str(part.text) for part in xml.iter('cx')]
        cy                = [str(part.text) for part in xml.iter('cy')]
        self.cX, self.cY  = float(cx[0]), float(cy[0])                   # Principal point in pixel coordinate system
        
        i_rX              = [str(part.get('width')) for part in xml.iter('resolution')]
        i_rY              = [str(part.get('height')) for part in xml.iter('resolution')]
        self.i_rX         = int(i_rX[0])
        self.i_rY         = int(i_rY[0])
        
        self.sensor_sizeX = self.pixel_size * self.i_rX
        self.sensor_sizeY = self.pixel_size * self.i_rY
        
        self.focalLength  = [str(part.text) for part in xml.iter('f')] 
        self.focal_length = float(self.focalLength[0]) * self.pixel_size
        self.iFov         = math.sqrt((self.sensor_sizeX*self.sensor_sizeY)/(self.i_rX*self.i_rY))/self.focal_length*1000
        
        self.VoxelK = np.zeros((ysize, xsize)).astype(int)
        for p in P_np:
            i, j, k = int(p[3]),int(p[4]),int(p[5])
            self.VoxelK[j,i] = k

        self.xstart, self.ystart = 0, 0
        self.xstop,  self.ystop  = xsize-1, ysize-1

    def printInfo(self):
        print("\n","[ Sensor Info","]")
        print("Sensor Nmae:",self.SensorID, "  Pixel Width:",self.i_rX, "  Pixel Hight:",self.i_rY)
        print("Pixel Size:",'{:.13f}'.format(self.pixel_size), end=" ")
        print("Cx:",'{:.5f}'.format(self.cX)," Cy:",'{:.5f}'.format(self.cY))
        print("Sensor SizeX(m)",'{:.5f}'.format(self.sensor_sizeX), "  Sensor SizeY(m)",'{:.5f}'.format(self.sensor_sizeY))
        print("Focal Length:",'{:.5f}'.format(float(self.focalLength[0])), " Camera iFOV:",'{:.3f}'.format(self.iFov))

## Original Image Class

In [19]:
class CameraParameterEach:
    def __init__(self, photoID):

        self.photoID = photoID
        
        # Input files
        self.IFile1 = Dir3 + photoID + "." + ImageExtension
        self.IFile2 = Dir2 + photoID + ".DNG"

        # Output files
        self.OFile1 = Dir4 + "VI-"           +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # Oltho
        self.OFile2 = Dir5 + "PA-"           +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # Phase Angle Image
        self.OFile3 = Dir4 + "OA-Luminunce-" +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # Luminunce
        self.OFile4 = Dir5 + "OA-RGB-"       +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # XYZ Normal using RGB
        self.OFile5 = Dir5 + "OA-Normal"     +photoID+ "-" +str(IntervalID)+ "-" +str(PixelArea)+ ".tif"  # XYZ Normal with 0 to 1


        # Reading the original image
        self.img  = tifffile.imread(self.IFile1)
        self.img_R,self.img_G,self.img_B = np.array(self.img[:,:,0]), np.array(self.img[:,:,1]), np.array(self.img[:,:,2])
        self.h, self.w, band             = self.img.shape

        # Camera parameters ###############################################################################
        self.num = camera_data_pd.loc[camera_data_pd['#PhotoID'] == photoID].index[0]
        self.x  = camera_data_pd["X"][self.num]
        self.y  = camera_data_pd["Y"][self.num]
        self.z  = camera_data_pd["Z"][self.num]
        self.Ci = round(((self.x-xmin)/Interval),0).astype(int)
        self.Cj = round(((ymax-self.y)/Interval),0).astype(int)
        self.Ck = round(((self.z-zmin)/Interval),0).astype(int)
        O       = camera_data_pd["Omega"][self.num]
        P       = camera_data_pd["Phi"][self.num]
        K       = camera_data_pd["Kappa"][self.num]
        
        # Degrees to Radians
        rad_O = math.radians(float(O))
        rad_P = math.radians(float(P))
        rad_K = math.radians(float(K))

        # Trigonometric Function Calculation
        sin_O = math.sin(-float(rad_O))
        cos_O = math.cos(-float(rad_O))
        sin_P = math.sin(-float(rad_P))
        cos_P = math.cos(-float(rad_P))
        sin_K = math.sin(-float(rad_K))
        cos_K = math.cos(-float(rad_K))
        
        # Collinearity equation
        arr2      = np.array([[cos_K,-sin_K,0], [sin_K,cos_K,0], [0,0,1]])  # kappa
        arr3      = np.array([[cos_P,0,sin_P], [0,1,0], [-sin_P,0,cos_P]])  # phi
        arr4      = np.array([[1,0,0], [0,cos_O,-sin_O], [0,sin_O,cos_O]])  # omega
        arr5      = np.dot(arr2, arr3)
        self.arr6 = np.dot(arr5, arr4)
                
        # Calculating the shooting range
        self.grand_reso   = siFov * (self.z - 135.00)   # The height of the target surface was assumed to be 135m.
        self.areaW, self.areaH = self.grand_reso * si_rX / 1000, self.grand_reso * si_rY / 1000
        self.iXmin, self.iXmax = self.x - (self.areaW/2), self.x + (self.areaW/2)
        self.iYmin, self.iYmax = self.y - (self.areaH/2), self.y + (self.areaH/2)

        #　Phase Angle Calculation  ###############################################################################
        transformer  = Transformer.from_crs("EPSG:2446", "EPSG:4326")

        # Get the shooting time
        Date = self.PhotoTakenTime(self.IFile2)
        self.Date = Date
        
        # Simulating the Sun
        Lat, lon             = transformer.transform(self.x, self.y)
        Lat_str, Lon_str     = str(Lat), str(lon)
        kochi                = ephem.Observer()
        kochi.lat, kochi.lon = Lat_str, Lon_str
        kochi.date           = Date
        kochi.date          -= ephem.hour * 9
        lt                   = ephem.localtime(kochi.date)
        sun                  = ephem.Sun(kochi)
        sun.compute(kochi)
        rhl                  = sun.hlon - ephem.degrees('180:00:00')
        dhl                  = ephem.degrees(rhl).norm
        sun_alt     = float(str(sun.alt).split(':')[0]) + float(str(sun.alt).split(':')[1]) / 60+float(str(sun.alt).split(':')[2]) / 3600
        sun_az      = float(str(sun.az).split(':')[0])  + float(str(sun.az).split(':')[1])  / 60+float(str(sun.az).split(':')[2])  / 3600
        self.sunalt = sun_alt # Solar altitude
        self.sunazl = sun_az  # Solar azimuth angle

        # Incident light direction vector　  ######################################################################
        rad_sun_alt = math.radians(sun_alt)
        rad_sun_azi = math.radians(sun_az)
        sx             = math.sin(rad_sun_azi) * math.cos(rad_sun_alt)
        sy             = math.cos(rad_sun_azi) * math.cos(rad_sun_alt)
        sz             = math.sin(rad_sun_alt)
        self.vectorSun = np.array([sx, sy, sz])

    # Obtaining the shooting time from the image data
    def PhotoTakenTime(self, PhotoFile):  
        exif_dict  = piexif.load(PhotoFile)
        for i,d in exif_dict["Exif"].items():
            if i == 36867:
                D = str(d)
        data       = (re.split('[: ]',D))
        Date  = data[0][2:6] +"/"+ data[1] +"/"+ data[2] +" "+ data[3] +":"+ data[4] +":"+ data[5][0:2]
        return Date

    def printInfo(self):
        print("\n","[ PHOTO INFO","]")
        print("PhotoID:",self.photoID, "  What line is written",self.num)
        print("x:",'{:.3f}'.format(self.x),"  y:",'{:.3f}'.format(self.y),"  z:",'{:.3f}'.format(self.z))
        print("Grand Resolution(mm)",'{:.3f}'.format(self.grand_reso), "  Width(m):",'{:.3f}'.format(self.areaW), "  Hight(m)",'{:.3f}'.format(self.areaH))
        print("::Image Range::"," Xmin(m)",'{:.3f}'.format(self.iXmin),"  Xmax(m)",'{:.3f}'.format(self.iXmax),"  Ymin(m)",'{:.3f}'.format(self.iYmin),"  Ymax(m)",'{:.3f}'.format(self.iYmax))
        print(self.sunalt ,self.sunazl)

## Functions: Coordinate conversion,Oblique,Phase angle,Observation angle and Save to GeoTiff 

In [20]:
# Coordinate transformation ###########################################################################
def convertXYZtoCR(workload):
    
    PX,PY,PZ =     workload[0] ,     workload[1] ,     workload[2]
    PI,PJ,PK = int(workload[3]), int(workload[4]), int(workload[5])
    R, G, B  = 0,0,0

    arr1  = np.array([[PX-cx], [PY-cy], [PZ-cz]])
    arr7  = np.dot(carr6, arr1)

    # Calculate UV from collinearity equation
    U   = -sFocalLength * (arr7[0,0] / arr7[2,0])
    V   = -sFocalLength * (arr7[1,0] / arr7[2,0])
    
    # Calculate Col Row
    col = round((U/sPixelSize)+iW/2+sCx,0).astype(int)
    row = round(iH/2-(V/sPixelSize)+sCy,0).astype(int)

    if 0<=col<iW and 0<=row<iH and 0<=PI<=xsize and 0<=PJ<=ysize :
            
            Rmin, Rmax = row-a, row+a+1
            Cmin, Cmax = col-a, col+a+1
            
            Rmin = 0      if Rmin < 0      else Rmin
            Rmax = iH     if Rmax > iH     else Rmax
            Cmin = 0      if Cmin < 0      else Cmin
            Cmax = iW     if Cmax > iW     else Cmax
            
            R, G, B = np.average(iR[Rmin:Rmax,Cmin:Cmax]), np.average(iG[Rmin:Rmax,Cmin:Cmax]), np.average(iB[Rmin:Rmax,Cmin:Cmax])

    return [PI, PJ, PK, R, G, B]


# Occlusion ###########################################################################
def DataFormatting1(OPa):
    OPb        = OPa[~np.all(OPa[:, 3:6] == 0, axis=1)]
    c_names   = ["PI", "PJ", "PK", "R", "G", "B"]
    DF        = pd.DataFrame(OPb,columns=c_names)
    DF["vi"]  = round(ci-DF["PI"],0).astype(int)
    DF["vj"]  = round(cj-DF["PJ"],0).astype(int)
    DF["vk"]  = round(ck-DF["PK"],0).astype(int)
    DF["d"]   = round(np.sqrt((DF["vi"])**2 + (DF["vj"])**2 + (DF["vk"])**2),0).astype(int)
    DF["T"]   = round(1 / DF["d"],8).astype(float)
    OPc       = DF.values
    return OPc
    
def Occlusion8PhaseAngle(workload):
    
    PI,PJ,PK = int(workload[0]) , int(workload[1]) , int(workload[2])
    R,G,B    =     workload[3]  ,     workload[4]  ,     workload[5]
    vi,vj,vk =     workload[6]  ,     workload[7] ,      workload[8]
    d, T     = int(workload[9]),     workload[10]

    ista, iend = PI if PI <= ci else ci, PI if PI >= ci else ci
    jsta, jend = PJ if PJ <= cj else cj, PJ if PJ >= cj else cj
    Tmax = np.max(VoxelK[jsta:jend+1, ista:iend+1]) 
    
    for n in range(0,d): 
        
        Tk, Tj, Ti = int(round(PK+vk*T*n,0)), int(round(PJ+vj*T*n,0)), int(round(PI+vi*T*n,0))

        if Tmax < Tk:
            break
        
        elif xstart>Ti or Ti>xstop or ystart>Tj or Tj>ystop :
            R,G,B = 0,0,0
            break

        moreHigher = VoxelK[Tj,Ti] > Tk
        if moreHigher==True:
            R,G,B = 0,0,0
            break

    return [PI, PJ, PK, R, G, B]


# Phase Angle ###########################################################################
def DataFormatting2(OPa):
    OPb        = OPa[~np.all(OPa[:, 3:6] == 0, axis=1)]
    c_names   = ["PI", "PJ", "PK", "R", "G", "B"]
    DF        = pd.DataFrame(OPb,columns=c_names)
    DF["px"]  = round(DF["PI"]*Interval+xmin,5).astype(float)
    DF["py"]  = round(ymax-DF["PJ"]*Interval,5).astype(float)
    DF["pz"]  = round(DF["PK"]*Interval+zmin,5).astype(float)
    DF        = DF.drop(["PK","R", "G", "B"],axis=1)
    OPc       = DF.values
    return OPc

def calculate_azimuth_elevation(x, y, z):

    r = math.sqrt(x**2 + y**2 + z**2)
    elevation = math.degrees(math.asin(z / r))

    if x == 0:
        if y > 0:
            return 0, elevation
        elif y < 0:
            return 180, elevation
    elif y == 0:
        if x > 0:
            return 90, elevation
        elif x < 0:
            return 270, elevation
    
    azimuth = math.atan2(x, y)
    azimuth = math.degrees(azimuth)
    azimuth = (azimuth + 360) % 360
    
    return azimuth,elevation

def PhaseAngle(workload):

    PI,PJ     = int(workload[0]) , int(workload[1])
    PX,PY,PZ =     workload[2] ,     workload[3] ,     workload[4]

    origin            = np.array([PX,PY,PZ])
    camera            = np.array([cx,cy,cz])

    vectorCam         = camera - origin

    dot_product       = np.dot(vectorCam,vectorSun)
    normSun           = np.linalg.norm(vectorSun)
    normCam           = np.linalg.norm(vectorCam)
    cosine_similarity = dot_product / (normCam*normSun)
    angle_in_radians  = np.arccos(cosine_similarity)
    angle_in_degrees  = np.degrees(angle_in_radians)

    #Observation Angle
    ox, oy, oz         = vectorCam
    azimuth, elevation = calculate_azimuth_elevation(ox, oy, oz)

    #Observation Angle To Normal
    Nxyz     =  np.linalg.norm(vectorCam)
    Normal   =  vectorCam / Nxyz
    Nx,Ny,Nz =  Normal

    return [PI, PJ, angle_in_degrees, azimuth, elevation, Nx, Ny, Nz]


# Geo Tiff ###########################################################################
def GeoTiffSave3(ResultList1,ResultList2,OFile): 

    imageP_np = np.zeros((ysize, xsize)).astype(float)

    imageR_np = np.zeros((ysize, xsize)).astype(int)
    imageG_np = np.zeros((ysize, xsize)).astype(int)
    imageB_np = np.zeros((ysize, xsize)).astype(int)

    for p in ResultList2:
        i, j    = int(p[0]),  int(p[1])
        PA      = int(p[2])
        imageP_np[j,i] = PA

    for p in ResultList1:
        i, j    = int(p[0]),  int(p[1])
        r, g, b = int(p[3]), int(p[4]), int(p[5])

        if imageP_np[j,i] == 0:
            imageR_np[j,i], imageG_np[j,i], imageB_np[j,i] = 0,0,0
        else:
            imageR_np[j,i], imageG_np[j,i], imageB_np[j,i] = r, g, b

    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile,xsize,ysize,3,gdal.GDT_UInt16)
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval)) 
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(2446)                                      # Japan Geodetic Survey 4th System
    dataset.SetProjection(srs.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(imageR_np)
    dataset.GetRasterBand(2).WriteArray(imageG_np)
    dataset.GetRasterBand(3).WriteArray(imageB_np)
    dataset.FlushCache()
    dataset = None

def GeoTiffSave1(ResultList,OFile): 

    imageP_np = np.zeros((ysize, xsize)).astype(float)

    for p in ResultList:
        i, j             = int(p[0]),  int(p[1])
        p                = p[2]
        imageP_np[j,i]   = p

    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile,xsize,ysize,1,gdal.GDT_Float32)
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(2446)                                      # Japan Geodetic Survey 4th System
    dataset.SetProjection(srs.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(imageP_np)
    dataset.FlushCache()
    dataset = None

def GeoTiffSave3A(ResultList1,ResultList2,OFile1,OFile2): 

    hue_np        = np.zeros((ysize, xsize)).astype(float)
    saturation_np = np.zeros((ysize, xsize)).astype(float)
    value_np      = np.zeros((ysize, xsize)).astype(float)

    for p in ResultList1:
        i, j                            = int(p[0]), int(p[1])
        phase_angle, azimuth, elevation = p[2], p[3], p[4]

        if elevation >= 70:
            hue                             = azimuth   / 360
            saturation                      = elevation / 90
            hue_np[j,i], saturation_np[j,i] = hue , saturation


    for p in ResultList2:
        i, j    = int(p[0]),  int(p[1])
        r, g, b = int(p[3]), int(p[4]), int(p[5])
        
        if hue_np[j,i] == 0:
            value_np[j,i] = 0
        else:
            value_np[j,i] = max(r, g, b) / 65535 # Divide wit Maximum number in 16 bit

    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile1,xsize,ysize,1,gdal.GDT_Float32)
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(2446)                                      # Japan Geodetic Survey 4th System
    dataset.SetProjection(srs.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(value_np)
    dataset.FlushCache()
    dataset = None

    R_np  = np.zeros((ysize, xsize)).astype(float)
    G_np  = np.zeros((ysize, xsize)).astype(float)
    B_np  = np.zeros((ysize, xsize)).astype(float)

    for j in range(0,ysize):
        for i in range(0,xsize):
            if value_np[j,i] > 0:
                R_np[j,i], G_np[j,i], B_np[j,i] = colorsys.hsv_to_rgb(hue_np[j,i],saturation_np[j,i],value_np[j,i])

    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile2,xsize,ysize,3,gdal.GDT_Float32)
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(2446)                                      # Japan Geodetic Survey 4th System
    dataset.SetProjection(srs.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(R_np)
    dataset.GetRasterBand(2).WriteArray(G_np)
    dataset.GetRasterBand(3).WriteArray(B_np)
    dataset.FlushCache()
    dataset = None


def GeoTiffSave3N(ResultList1,OFile): 

    imageNx_np   = np.zeros((ysize, xsize)).astype(float)
    imageNy_np   = np.zeros((ysize, xsize)).astype(float)
    imageNz_np   = np.zeros((ysize, xsize)).astype(float)

    for p in ResultList1:
        i, j      = int(p[0]),  int(p[1])
        elevation = p[4]
        Nx,Ny,Nz  = p[5],p[6],p[7]

        if elevation >= 70:
            imageNx_np[j,i] = np.round(((Nx + 1)/2) * 255,0)
            imageNy_np[j,i] = np.round(((Ny + 1)/2) * 255,0)
            imageNz_np[j,i] = np.round(((Nz + 1)/2) * 255,0)

    driver  = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(OFile,xsize,ysize,3,gdal.GDT_Byte)
    dataset.SetGeoTransform((xmin,Interval,0,ymax,0,-Interval))
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(2446)                                      # Japan Geodetic Survey 4th System
    dataset.SetProjection(srs.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(imageNx_np)
    dataset.GetRasterBand(2).WriteArray(imageNy_np)
    dataset.GetRasterBand(3).WriteArray(imageNz_np)
    dataset.FlushCache()
    dataset = None

# Parallel Processing

In [21]:
# Workload division
def Workers(P_np):
    number , column = P_np.shape
    workers      = int(os.cpu＿count())
    workloadRow  = number//workers  # 切り下げ
    return number, workers, workloadRow


# For Coordinate transformation ################################################################
def parallelprocessing(P_np, number, workers, workloadRow, wnumber):
    start      = workloadRow * wnumber
    end        = number + 1 if wnumber == workers else workloadRow * (wnumber + 1)
    workload   = P_np[start:end]
    resultlist = list(map(convertXYZtoCR,workload))
    return resultlist

def Confu(self):
    with confu.ProcessPoolExecutor(max_workers=os.cpu＿count()) as executor: 
        number, workers, workloadRow = Workers(P_np)
        futures = [executor.submit(parallelprocessing, P_np, number, workers, workloadRow, wnumber) for wnumber in range(0,workers)]
        marge_PointList = np.array([[0,1,2,3,4,5]])
        for f in tqdm(futures):
            result_XYZUVIJK = f.result()
            marge_PointList  = np.concatenate([marge_PointList, result_XYZUVIJK],0)

    PointsList2Bsaved = np.delete(marge_PointList, 0, axis=0)
    return PointsList2Bsaved


# For Occlusion ###############################################################################
def parallelprocessing_2(OP2, number, workers, workloadRow, wnumber):
    start      = workloadRow * wnumber
    end        = number + 1 if wnumber == workers else workloadRow * (wnumber + 1)
    workload   = OP2[start:end]
    resultlist = list(map(Occlusion8PhaseAngle,workload))
    return resultlist

def Confu_2(OP2):
    with confu.ProcessPoolExecutor(max_workers=os.cpu＿count()) as executor: 
        number, workers, workloadRow = Workers(OP2)
        futures = [executor.submit(parallelprocessing_2, OP2, number, workers, workloadRow, wnumber) for wnumber in range(0,workers)]
        marge_PointList = np.array([[0,1,2,3,4,5]])
        for f in tqdm(futures):
            result_IJKRGB = f.result()
            marge_PointList  = np.concatenate([marge_PointList, result_IJKRGB],0)

    PointsList2Bsaved = np.delete(marge_PointList, 0, axis=0)
    return PointsList2Bsaved


# For Calculate Angle #############################################################################
def parallelprocessing_3(OP4, number, workers, workloadRow, wnumber):
    start      = workloadRow * wnumber
    end        = number + 1 if wnumber == workers else workloadRow * (wnumber + 1)
    workload2  = OP4[start:end]
    resultlist = list(map(PhaseAngle,workload2))
    return resultlist

def Confu_3(OP4):
    with confu.ProcessPoolExecutor(max_workers=os.cpu＿count()) as executor: 
        number, workers, workloadRow = Workers(OP4)
        futures = [executor.submit(parallelprocessing_3, OP4, number, workers, workloadRow, wnumber) for wnumber in range(0,workers)]
        marge_PointList = np.array([[0,1,2,3,4,5,6,7]])
        for f in tqdm(futures):
            result_IJPa = f.result()
            marge_PointList  = np.concatenate([marge_PointList, result_IJPa],0)

    PointsList2Bsaved = np.delete(marge_PointList, 0, axis=0)
    return PointsList2Bsaved

# Program execution algorithm

In [22]:
# SensorParameter
Sensor = SensorParameter(CameraName)

sPixelSize   = Sensor.pixel_size
sFocalLength = Sensor.focal_length
sCx , sCy    = Sensor.cX, Sensor.cY
siFov        = Sensor.iFov
si_rX, si_rY = Sensor.i_rX,Sensor.i_rY
VoxelK       = Sensor.VoxelK
xstart, ystart, xstop, ystop = Sensor.xstart, Sensor.ystart, Sensor.xstop, Sensor.ystop

Sensor.printInfo()
print(CameraName)

for i in PhotoIDs:
    start  = time.time()
    print(i,end=" ")

    Image                = CameraParameterEach(i)
    iH, iW               = int(Image.h), int(Image.w)
    iR, iG, iB           = Image.img_R,Image.img_G,Image.img_B
    cx, cy, cz           = Image.x, Image.y, Image.z
    ci, cj, ck           = Image.Ci, Image.Cj, Image.Ck
    carr6                = Image.arr6
    vectorSun            = Image.vectorSun
    OFile1,OFile2,OFile3 = Image.OFile1, Image.OFile2, Image.OFile3
    OFile4,OFile5        = Image.OFile4, Image.OFile5
    print(Image.sunalt, Image.sunazl, Image.vectorSun)

    if xstart > ci or ci > xstop or ystart > cj or cj > ystop:
        df = pd.read_csv(OFile1, header=None)
        df = df[df[0] != i]
        df.to_csv(OFile1, index=False, header=False)
        print(": outside the area , remove from list")
        continue

    # Orthophoto Creation
    OP1 = Confu(P_np) 

    # Occlusion Processing
    OP2 = DataFormatting1(OP1)
    OP3 = Confu_2(OP2)

    # Calculating angles
    OP4 = DataFormatting2(OP3)
    OP5 = Confu_3(OP4)

    # SaveImage # Output only the parts where all values are correct
    GeoTiffSave3(OP3,OP5,OFile1)             # Oltho Photo
    GeoTiffSave1(OP5,OFile2)                 # Phase Angle
    GeoTiffSave3A(OP5,OP3,OFile3,OFile4)     # Observation Angle and Luminance
    GeoTiffSave3N(OP5,OFile5)                # Observation Angle Blue Map

    del OP1
    del OP2
    del OP3
    del OP4
    del OP5

    print ("elapsed_time:{0}".format(time.time()-start) + "[sec]")


 [ Sensor Info ]
Sensor Nmae: P1   Pixel Width: 8192   Pixel Hight: 5460
Pixel Size: 0.0000043948614 Cx: -34.02808  Cy: 21.09109
Sensor SizeX(m) 0.03600   Sensor SizeY(m) 0.02400
Focal Length: 7973.36592  Camera iFOV: 0.125
P1
DJI_20231228115449_0106 33.43872222222222 177.26816666666667 [ 0.0397723  -0.8335273   0.55104483]


100%|██████████| 64/64 [00:02<00:00, 26.31it/s]


elapsed_time:31.56963849067688[sec]
